In [1]:
from __future__ import print_function, division
import os
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from utils import *
from utils.KittiColormap import *
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from datasets.data_io import get_transform, read_all_lines, pfm_imread
from PIL import Image
from tqdm import tqdm, trange
from models.mobilestereonet.models.MSNet2D import MSNet2D
from models.mobilestereonet.models.MSNet3D import MSNet3D
from models.Voxel2D import Voxel2D
from sklearn.metrics import accuracy_score, f1_score
import coloredlogs, logging
from datasets import VoxelDSDataset
import torch.nn.functional as F
import traceback
from torchmetrics.functional import jaccard_index
from pytorch3d.loss import chamfer_distance

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
left_img = torch.rand((1,3, 400, 880))
right_img = torch.rand((1,3, 400, 880))

In [ ]:
model = MSNet2D(192)
model = nn.DataParallel(model)
ckpt_path = "../models/MSNet2D_SF_DS.ckpt"
print("Loading model {}".format(ckpt_path))
state_dict = torch.load(ckpt_path, map_location="cpu")
model.load_state_dict(state_dict['model'])

In [ ]:
model.cpu()
model.eval()

In [ ]:
%timeit -n 2 -r 5 model(left_img, right_img)

In [ ]:
from models.CFNet.models import CFNet
model = CFNet(192)
model = nn.DataParallel(model)
ckpt = torch.load("models/CFNet/finetuning_model", map_location="cpu")
model.load_state_dict(ckpt['model'])
model.cpu()
model.eval()

In [ ]:
left_img = torch.rand((1,3, 384, 864))
right_img = torch.rand((1,3, 384, 864))

In [ ]:
%timeit -n 2 -r 5 model(left_img, right_img)

In [ ]:
from models.ACVNet.models import ACVNet
model = ACVNet(192, False, False)

model = nn.DataParallel(model)
ckpt = torch.load("models/ACVNet/pretrained_model/checkpoint_000009.ckpt", map_location="cpu")
model.load_state_dict(ckpt["model"])
model.cpu()
model.eval()

In [ ]:
%timeit -n 2 -r 5 model(left_img, right_img)

In [ ]:
from models.lacGwcNet.networks.stackhourglass import PSMNet
# load model
affinity_settings = {}
affinity_settings['win_w'] = 3
affinity_settings['win_h'] = 3
affinity_settings['dilation'] = [1, 2, 4, 8]
model = PSMNet(maxdisp=192, struct_fea_c=4, fuse_mode="separate",
           affinity_settings=affinity_settings, udc=True, refine="csr")

model = nn.DataParallel(model)
ckpt = torch.load("models/lacGwcNet/checkpoint_9.tar", map_location="cpu")
model.load_state_dict(ckpt["net"])
model.cpu()
model.eval()

In [ ]:
%timeit -n 2 -r 5 model(left_img, right_img, None)

In [3]:
import cv2
#Note: disparity range is tuned according to specific parameters obtained through trial and error. 
win_size = 5
min_disp = -1
max_disp = 191 #min_disp * 9
num_disp = max_disp - min_disp # Needs to be divisible by 16
#Create Block matching object. 
sgbm = cv2.StereoSGBM_create(minDisparity= min_disp,
numDisparities = num_disp,
blockSize = 5,
uniquenessRatio = 5,
speckleWindowSize = 5,
speckleRange = 5,
disp12MaxDiff = 1,
P1 = 8*3*win_size**2,#8*3*win_size**2,
P2 =32*3*win_size**2) #32*3*win_size**2)

list_filename = "./filenames/DS_test.txt"
lines = read_all_lines(list_filename)
splits = [line.split() for line in lines]
left_filenames = [x[0] for x in splits]
right_filenames = [x[1] for x in splits]
disp_filenames = [x[2] for x in splits]

def load_image(filename):
    return Image.open(filename).convert('RGB')

def load_disp(filename):
    data = Image.open(filename)
    data = np.array(data, dtype=np.float32) / 256.
    return data

data_index = 9
datapath = "/home/chris/pl_ws/src/stereo_pl_nav/datasets/DS"
left_img = load_image(os.path.join(datapath, left_filenames[data_index]))
right_img = load_image(os.path.join(datapath, right_filenames[data_index]))
disparity = load_disp(os.path.join(datapath, disp_filenames[data_index]))
left_img = np.asarray(left_img)
right_img = np.asarray(right_img)
left_gray = cv2.cvtColor(left_img, cv2.COLOR_RGB2GRAY)
right_gray = cv2.cvtColor(right_img, cv2.COLOR_RGB2GRAY)

In [4]:
%timeit -n 10 -r 5 sgbm.compute(left_gray,right_gray)

211 ms ± 380 µs per loop (mean ± std. dev. of 5 runs, 10 loops each)


In [ ]:
from models.Voxel2D import Voxel2D
model = Voxel2D(192, "voxel")
model = nn.DataParallel(model)
ckpt_path = "/home/chris/pl_ws/src/stereo_pl_nav/scripts/voxelstereonet/logs/lr_0.001_batch_size_16_cost_vol_type_voxel_optimizer_adam_/best.ckpt"
state_dict = torch.load(ckpt_path, map_location="cpu")
model.load_state_dict(state_dict['model'])
model.cpu()
model.eval()

In [ ]:
# Camera intrinsics and extrinsics
c_u = 4.556890e+2
c_v = 1.976634e+2
f_u = 1.003556e+3
f_v = 1.003556e+3
b_x = 0.0
b_y = 0.0
baseline = 0.54
# calculate voxel cost volume disparity set
vox_cost_vol_disp_set = set()
max_disp = 192
# depth starting from voxel_size since 0 will cause issue
for z in np.arange(0.5, 32, 2.0):
    # get respective disparity
    d = f_u * baseline / z

    if d > max_disp:
        continue

    # real disparity -> disparity in feature map
    vox_cost_vol_disp_set.add(round(d/4))

vox_cost_vol_disps = list(vox_cost_vol_disp_set)
vox_cost_vol_disps = sorted(vox_cost_vol_disps)

tmp = []
for i in vox_cost_vol_disps:
    tmp.append(torch.unsqueeze(torch.Tensor([i]), 0))
vox_cost_vol_disps = tmp

In [ ]:
len(vox_cost_vol_disps)

In [ ]:
left_img = torch.rand((1,3, 400, 880))
right_img = torch.rand((1,3, 400, 880))
%timeit -n 2 -r 5 model(left_img, right_img, vox_cost_vol_disps)

In [ ]:
from models.Voxel2D_lite import Voxel2D
model = Voxel2D(192, "voxel")
model = nn.DataParallel(model)
ckpt_path = "/home/chris/pl_ws/src/stereo_pl_nav/scripts/voxelstereonet/logs/lr_0.001_batch_size_32_cost_vol_type_voxel_optimizer_adam_lite/best.ckpt"
state_dict = torch.load(ckpt_path, map_location="cpu")
model.load_state_dict(state_dict['model'])
model.cpu()
model.eval()

In [ ]:
# Camera intrinsics and extrinsics
c_u = 4.556890e+2
c_v = 1.976634e+2
f_u = 1.003556e+3
f_v = 1.003556e+3
b_x = 0.0
b_y = 0.0
baseline = 0.54
# calculate voxel cost volume disparity set
vox_cost_vol_disp_set = set()
max_disp = 192
# depth starting from voxel_size since 0 will cause issue
for z in np.arange(0.5, 32, 0.5):
    # get respective disparity
    d = f_u * baseline / z

    if d > max_disp:
        continue

    # real disparity -> disparity in feature map
    vox_cost_vol_disp_set.add(round(d/8))

vox_cost_vol_disps = list(vox_cost_vol_disp_set)
vox_cost_vol_disps = sorted(vox_cost_vol_disps)

tmp = []
for i in vox_cost_vol_disps:
    tmp.append(torch.unsqueeze(torch.Tensor([i]), 0))
vox_cost_vol_disps = tmp

In [ ]:
len(vox_cost_vol_disps)

In [ ]:
left_img = torch.rand((1,3, 400, 880))
right_img = torch.rand((1,3, 400, 880))
%timeit -n 2 -r 5 model(left_img, right_img, vox_cost_vol_disps)